In [9]:
from nba_api.stats.static import players
from nba_api.stats.endpoints import PlayerGameLog
import pandas as pd
import nba_api.stats.endpoints

In [13]:
draft_df = nba_api.stats.endpoints.drafthistory.DraftHistory().get_data_frames()[0]

In [ ]:
draft_2003 = draft_df[draft_df["SEASON"] == "2003"]
draft_2003

In [ ]:
players_name_id = draft_2003[["PLAYER_NAME", "PERSON_ID"]]

In [29]:
players_10_games = []
for player_id, player_name in zip(players_name_id["PERSON_ID"], players_name_id["PLAYER_NAME"]):
	game_log = PlayerGameLog(player_id=player_id, season="2003-04", season_type_all_star="Regular Season")
	df = game_log.get_data_frames()[0]
	df["GAME_DATE"] = pd.to_datetime(df["GAME_DATE"])
	df_sorted = df.sort_values("GAME_DATE")
	first_10_games = df_sorted.head(10)
	player_stats = {
		"player_name": player_name,
		"player_id": player_id,
		"fta": first_10_games["FTA"].sum(),
		"ftm": first_10_games["FTM"].sum(),
		"ft_pct": first_10_games["FTM"].sum() / first_10_games["FTA"].sum() if first_10_games["FTA"].sum() > 0 else 0
	}
	players_10_games.append(player_stats)

df_players_10_games = pd.DataFrame(players_10_games)

In [30]:
df_players_10_games

,player_name,player_id,fta,ftm,ft_pct
0,LeBron James,2544,38,23,0.605263
1,Darko Milicic,2545,3,1,0.333333
2,Carmelo Anthony,2546,60,43,0.716667
3,Chris Bosh,2547,27,23,0.851852
4,Dwyane Wade,2548,36,24,0.666667
5,Chris Kaman,2549,17,12,0.705882
6,Kirk Hinrich,2550,22,15,0.681818
7,T.J. Ford,2551,17,15,0.882353
8,Michael Sweetney,2552,6,5,0.833333
9,Jarvis Hayes,2553,11,9,0.818182
